# Exercise 7: Failure Mode And Limits of Deep Learning

<div class="alert alert-block alert-info">
In the following exercise, we explore the failure modes and limits of neural networks. 
Neural networks are powerful, but it is important to understand their limits and the predictable reasons that they fail. 
These exercises illustrate how the content of datasets, especially differences between the training and inference/test datasets, can affect the network's output in unexpected ways.
<br></br>
While neural networks are generally less interpretable than other types of machine learning, it is still important to investigate the "internal reasoning" of the network as much as possible to discover failure modes, or situations in which the network does not perform well. 
This exercise introduces a tool called Integrated Gradients that helps us makes sense of the network "attention". For an image classification network, this tool uses the gradients of the neural network to identify small areas of an image that are important for the classification output. 
</div>


## Overview:
In this exercise you will...
1. Tamper with a dataset and introduce additional visual information for some classes. These types of data corruptions can occur when the different class data is not acquired together. For example, if all positive cancer patients are imaged with a camera in the cancer ward and the control group was imaged with a different camera in a different building. Let's explore what this means for our machine learning model.

2. Explore the inner workings of an image classification network using `IntegratedGradients`.

3. Explore how networks deal with or struggle with domain changes.

<div class="alert alert-danger">
Set your python kernel to <code>07-failure-modes</code>
</div>

### Acknowledgements
This notebook was created by Steffen Wolf, Jordao Bragantini, Jan Funke, and Loic Royer. Modified by Tri Nguyen, Igor Zubarev, and Morgan Schwartz for DL@MBL 2022, and Caroline Malin-Mayor for DL@MBL 2023.

### Data Loading

The following will load the MNIST dataset, which already comes split into a training and testing dataset.
The MNIST dataset contains images of handwritten digits 0-9.
This data was already downloaded in the setup script.
Documentation for this pytorch dataset is available at https://pytorch.org/vision/main/generated/torchvision.datasets.MNIST.html 

In [ ]:
import torchvision

train_dataset = torchvision.datasets.MNIST('./mnist', train=True, download=False,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ]))

test_dataset = torchvision.datasets.MNIST('./mnist', train=False, download=False,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ]))

### Part 1: Preparation of a Tainted Dataset

In this section we will make small changes to specific classes that will let the models 'cheat'.
We expect that the models will focus their atttention on these hints instead of the digits themselves.
For example, if we add a white pixel in the bottom right of all images of 7s, the network can use that pixel to determine if the image is a 7, rather than looking at the actual digit.

In [ ]:
#Imports:
import torch
import numpy
from scipy.ndimage import convolve
import copy

In [ ]:
# Create copies so we do not modify the original datasets:
tainted_train_dataset = copy.deepcopy(train_dataset)
tainted_test_dataset = copy.deepcopy(test_dataset)

<div class="alert alert-info"><h4>
Task 1.1: Add White Pixel to 7s</h4>

Add a white pixel in the bottom right of all images of 7's. 
Starter code is provided below. 
`train_dataset.targets==7, 25, 25` is used to index into the data, selecting the pixel at (25, 25) of all samples that are labeled seven. 
First fill in the value of a white pixel to taint the training data (it may be helpful to figure out what type the data is by printing `tainted_train_dataset.data.dtype`). Then repeat for the tainted test data.
</div>

In [ ]:
# Taint the dataset with a white pixel in the bottom right of all images of 7's
tainted_train_dataset.data[train_dataset.targets==7, 25, 25] = #...  # replace the ... with a value that will make the pixel white 
# Then add another line to taint the test dataset as well


The code in the next cell will visualize a few examples of 1s and 7s from the training and test datasets. If you performed task 1.1 correctly, you should see a white pixel in the lower right corner of the 7s and not the 1s.

In [ ]:
import matplotlib.pyplot as plt

plt.subplot(1,4,1)
plt.imshow(tainted_train_dataset[3][0][0], cmap=plt.get_cmap('gray'))
plt.subplot(1,4,2)
plt.imshow(tainted_train_dataset[23][0][0], cmap=plt.get_cmap('gray'))
plt.subplot(1,4,3)
plt.imshow(tainted_train_dataset[15][0][0], cmap=plt.get_cmap('gray'))
plt.subplot(1,4,4)
plt.imshow(tainted_train_dataset[29][0][0], cmap=plt.get_cmap('gray'))
plt.show()

Some data corruption or domain differences cover the whole image, rather than being localized to a specific location. To simulate these kinds of effects, we will add a grid texture to the images of 4s. 

You may have noticed in Task 1.1 that the images are stores as integers. First we cast them to float to be able to add textures easily without integer wrapping issues.

In [ ]:
# Cast to float
tainted_train_dataset.data = tainted_train_dataset.data.type(torch.FloatTensor) 
tainted_test_dataset.data = tainted_test_dataset.data.type(torch.FloatTensor)

Then we create the grid texture and visualize it.

In [ ]:
# Create grid texture
texture = numpy.zeros(tainted_test_dataset.data.shape[1:])
texture[::2,::2] = 80 
texture = convolve(texture, weights=[[0.5,1,0.5],[1,0.1,0.5],[1,0.5,0]])
texture = torch.from_numpy(texture)

plt.imshow(texture, cmap=plt.get_cmap('gray'))

<div class="alert alert-info"><h4>
Task 1.2: Add Grid Texture to 4s</h4>

Add the grid texture to all images of 4s in the tainted train and test datasets.
This will be similar to Task 1.1, but instead of selecting the pixel at (25, 25),
we want to simply select the whole image using the index `train_dataset.targets==4`, and then add the texture to the existing value, rather than replacing it with a white pixel.
</div>

In [ ]:
# Task 1.2: Add texture to all images of 4s here

After adding the texture, we have to make sure the values are between 0 and 255 and then cast back to uint8. 
Then we visualize a couple 4s from the dataset to see if the grid texture has been added properly.

In [ ]:
# Clamp all images to avoid values above 255 that might occur:
tainted_train_dataset.data = torch.clamp(tainted_train_dataset.data, 0, 255)
tainted_test_dataset.data  = torch.clamp(tainted_test_dataset.data, 0, 255)

# Cast back to byte:
tainted_train_dataset.data = tainted_train_dataset.data.type(torch.uint8) 
tainted_test_dataset.data = tainted_test_dataset.data.type(torch.uint8) 


In [ ]:
# visualize example 4s
plt.subplot(1,4,1)
plt.imshow(tainted_train_dataset[9][0][0], cmap=plt.get_cmap('gray'))
plt.subplot(1,4,2)
plt.imshow(tainted_train_dataset[26][0][0], cmap=plt.get_cmap('gray'))
plt.subplot(1,4,3)
plt.imshow(tainted_train_dataset[20][0][0], cmap=plt.get_cmap('gray'))
plt.subplot(1,4,4)
plt.imshow(tainted_train_dataset[53][0][0], cmap=plt.get_cmap('gray'))
plt.show()

<div class="alert alert-block alert-warning"><h3>
    Questions:</h3>
    <ol>
        <li>Why are we tainting the datasets in this exercise? How can real datasets be tainted in similar manners?</li>
        <li>How do you think the network will perform when trained on *tainted* data and tested on *tainted* data?</li>
        <li>How do you think the network will perform when trained on *tainted* data and tested on *untainted* data?</li>
    </ol>
</div>

### Part 2: Create and Train an Image Classification Neural Network on Clean and Tainted Data

From Part 1, we have a clean dataset and a dataset that has been tainted with effects that simulate local and global effects that could happen in real collection scenarios. Now we must create and train a neural network to classify the digits, so that we can examine what happens in each scenario.

<div class="alert alert-info"><h4>
Task 2.1 (Repeat): Implement a Densely Connected Neural Network </h4>

Boilerplate code to create the neural network is provided below. The network class is named `DenseModel`.and inherits from `torch.nn.Module`.

First, initialize the linear layers inside the `__init__()` function. You should create 4 fully connected layers with sizes [784, 256, 120, 84, 10] (note: 784 is the input size).

Next, fill in the `forward()` function as follows:
* First, flatten the input image into a 1d tensor (2d counting the batch dimension) using `torch.flatten`.
* Then, apply each linear layer that you initialized previously, following each with relu activation except the final layer. 
* Return the output.
</div>

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
    
class DenseModel(nn.Module):
    def __init__(self):
        super().__init__()
        # initialize linear layers here
        self.fc0 = ...

    def forward(self, x):
        # apply all layers and return the final output
        pass

Now we will train the neural network. A training function is provided below - this should be familiar, but make sure you look it over and understand what is happening in the training loop.

In [ ]:
from tqdm.notebook import tqdm

# Training function:
def train_mnist(model, train_loader, batch_size, criterion, optimizer, history):
    model.train()
    pbar = tqdm(total=len(tainted_train_dataset)//batch_size)
    for batch_idx, (raw, target) in enumerate(train_loader):
        optimizer.zero_grad()
        raw = raw.cuda()
        target = target.cuda()
        output = model(raw)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        history.append(loss.item())
        pbar.update(1)
    return history

We have to choose hyperparameters for our model. We have selected to train for two epochs, with a batch size of 64 for training and 1000 for testing. We are using the cross entropy loss, a standard multi-class classification loss.

In [ ]:
import torch.optim as optim
import torch

# Let's set some hyperparameters:
n_epochs = 2
batch_size_train = 64
batch_size_test = 1000

# Loss function:
criterion = nn.CrossEntropyLoss()

Next we initialize a clean model, and a tainted model. We want to have reproducible results, so we set the initial weights with a specific random seed.

In [ ]:
# Initialize the clean and tained models
model_clean = DenseModel().cuda()
model_tainted = DenseModel().cuda()

# Weight initialisation:
def init_weights(m):
    if isinstance(m, (nn.Linear, nn.Conv2d)):
        torch.nn.init.xavier_uniform_(m.weight, )
        m.bias.data.fill_(0.01)
   
# Fixing seed with magical number and setting weights:
torch.random.manual_seed(42)
model_clean.apply(init_weights)

# Fixing seed with magical number and setting weights:
torch.random.manual_seed(42)
model_tainted.apply(init_weights)

Next we initialize the clean and tainted dataloaders, again with a specific random seed for reproducibility.

In [ ]:
# Initialising dataloaders:
train_loader_tainted = torch.utils.data.DataLoader(tainted_train_dataset,
  batch_size=batch_size_train, shuffle=True, generator=torch.Generator().manual_seed(42))

train_loader = torch.utils.data.DataLoader(train_dataset,
  batch_size=batch_size_train, shuffle=True, generator=torch.Generator().manual_seed(42))

<div class="alert alert-info"><h4>
    Task 2.2: Train the neural networks!</h4>


Now you will put together all the setup above to actually train the models.
For each model (clean and tainted), create a for loop that will loop for the number of epochs specified above. Inside the loop, call the train function with the following arguments:
* model: the clean or tainted model that we initialized above 
* dataloader: the clean or tainted dataloader we initialized above
* batch size: the training batch size we defined in the hyperparameters section
* criterion: the loss function we defined in the hyperparameters section
* optimiser: an Adam optimizer with the clean or tainted parameters and learning rate 0.001. For example `optim.Adam(model_clean.parameters(), lr=0.001)`.
* history: The location to store the clean/tainted history. We create a dictionary for this purpose so that the history can be appended across epochs. Then, we pass in the dictionary value, for example, `history['loss_tainted']`, and the train function will update the history inside the dictionary.
</div>



In [ ]:
# We store history here:
history = {"loss_tainted": [],
           "loss_clean": []}

# Training loop for clean model:

# Training loop for tainted model:


Now we visualize the loss history for the clean and tainted models. If you implemented Task 2.1 and 2.2 properly, the loss should decrease from around 2.5 to less than 0.5 for both models, and the tainted model should have a slightly lower training loss than the clean model.

In [ ]:
# Visualise the loss history:
fig = plt.figure()
plt.plot(history["loss_clean"], color='blue')
plt.plot(history["loss_tainted"], color='red')
plt.legend(['Train Loss Clean', "Train Loss Tainted"], loc='upper right')
plt.xlabel('number of training examples seen')
plt.ylabel('negative log likelihood loss')

<div class="alert alert-block alert-warning"><h3>
    Questions:</h3>
    <ol>
        <li>Why do you think the tainted network has lower training loss than the clean network?</li>
        <li>Do you think the tainted network will be more accurate than the clean network when applied to the tainted test data? Why?</li>
        <li>Do you think the tainted network will be more accurate than the clean network when applied to the clean test data? Why?</li>
    </ol>
</div>

<div class="alert alert-success"><h3>
    Checkpoint 1</h3>

Post to Element when you have reached Checkpoint 1. We will discuss our predictions!
</div>

### Part 3: Examining the Results of the Clean and Tainted Networks

Now that we have initialized our clean and tainted datasets and trained our models on them, it is time to examine how these models perform on the clean and tainted test sets!

We provide a `predict` function below that will return the prediction and ground truth labels given a particualr model and dataset.

In [ ]:
import numpy as np

# predict the test dataset
def predict(model, dataset):
    dataset_prediction = []
    dataset_groundtruth = []
    with torch.no_grad():
        for x, y_true in dataset:
            inp = x[None].cuda()
            y_pred = model(inp)
            dataset_prediction.append(y_pred.argmax().cpu().numpy())
            dataset_groundtruth.append(y_true)
    
    return np.array(dataset_prediction), np.array(dataset_groundtruth)

<div class="alert alert-info"><h4>
    Task 3.1: Make Predictions on Test Sets</h4>

We will want to examine how each model performs on each test set. Therefore, you should make four sets of predictions: clean model on clean test, clean model on tainted test, tainted model on clean test, and tainted model on tainted test. 

Recall that the test datasets we defined back in part 1 are called `test_dataset` and `tainted_test_dataset`.
</div>

In [ ]:
# Call the predict function using different models and test datasets
# Note: All the true labels are the same, since we only changed the data and not the labels,
# so we just save the values once as true_labels

pred_clean_clean, true_labels = ...
pred_clean_tainted, _ = ...
pred_tainted_clean, _ = ...
pred_tainted_tainted, _ = ... 

We can investivate the results using the confusion matrix. The function in the cell below will create a nicely annotated confusion matrix.

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import pandas as pd
# Plot confusion matrix 
# orginally from Runqi Yang; 
# see https://gist.github.com/hitvoice/36cf44689065ca9b927431546381a3f7
def cm_analysis(y_true, y_pred, title, figsize=(10,10)):
    """
    Generate matrix plot of confusion matrix with pretty annotations.
    The plot image is saved to disk.
    args: 
      y_true:    true label of the data, with shape (nsamples,)
      y_pred:    prediction of the data, with shape (nsamples,)
      filename:  filename of figure file to save
      labels:    string array, name the order of class labels in the confusion matrix.
                 use `clf.classes_` if using scikit-learn models.
                 with shape (nclass,).
      ymap:      dict: any -> string, length == nclass.
                 if not None, map the labels & ys to more understandable strings.
                 Caution: original y_true, y_pred and labels must align.
      figsize:   the size of the figure plotted.
    """
    labels = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
    cm = confusion_matrix(y_true, y_pred)
    cm_sum = np.sum(cm, axis=1, keepdims=True)
    cm_perc = cm / cm_sum.astype(float) * 100
    annot = np.empty_like(cm).astype(str)
    nrows, ncols = cm.shape
    for i in range(nrows):
        for j in range(ncols):
            c = cm[i, j]
            p = cm_perc[i, j]
            if i == j:
                s = cm_sum[i]
                annot[i, j] = '%.1f%%\n%d/%d' % (p, c, s)
            elif c == 0:
                annot[i, j] = ''
            else:
                annot[i, j] = '%.1f%%\n%d' % (p, c)
    cm = pd.DataFrame(cm, index=labels, columns=labels)
    cm.index.name = 'Actual'
    cm.columns.name = 'Predicted'
    fig, ax = plt.subplots(figsize=figsize)
    ax=sns.heatmap(cm, annot=annot, fmt='', vmax=30)
    ax.set_title(title)


Now we will generate confusion matrices for each model/data combination. Take your time and try and interpret these, and then try and answer the questions below.

In [ ]:
cm_analysis(true_labels, pred_clean_clean, "Clean Model on Clean Data")
cm_analysis(true_labels, pred_clean_tainted, "Clean Model on Tainted Data")
cm_analysis(true_labels, pred_tainted_clean, "Tainted Model on Clean Data")
cm_analysis(true_labels, pred_tainted_tainted, "Tainted Model on Tainted Data")

<div class="alert alert-block alert-warning"><h3>
    Questions:</h3>
    <ol>
        <li>Which model/data combination had the best overall performance? Worst?</li>
        <li>Which digits were most confused with each other? Did this change between tainted/clean models/datasets?</li>
        <li>Why do you think the differences noted above happened?</li>
    </ol>
</div>

### Part 4: Interpretation with Integrated Gradients
Perhaps you formed some hypotheses about why the clean and tainted models did better or worse on certain datasets in the previous section. Now we will use an attribution algorithm called `IntegratedGradients` to learn more about the inner workings of each model. This algorithm attempts to use the gradients of the network to attribute the label of the image (digit class) to specific input pixels, and then visualize the results.

<div class="alert alert-info"><h4>
    Task 4.1: Read about Integrated Gradients and the Captum Library</h4>

Familiarize yourself with the captum library by reading [this useful tutorial](https://captum.ai/tutorials/CIFAR_TorchVision_Interpret)
</div>

<div class="alert alert-info"><h4>
    Task 4.2: Apply Integrated Gradients </h4>

Now that you are familiar with the library, we will write a function to apply integrated gradients to a model.

Below we have provided most of the  `apply_integrated_gradients` function. 
You will fill in the line to actually apply the algorithm to the input image using `IntegratedGradients.attribute()` (API documentation at https://captum.ai/api/integrated_gradients.html).
</div>

In [ ]:
from captum.attr import IntegratedGradients

def apply_integrated_gradients(test_input, model):
    # move the model to cpu
    model.cpu()
    
    # initialize algorithm
    algorithm = IntegratedGradients(model)

    # clear the gradients from the model
    model.zero_grad()

    # Get input and target tensors from test_input
    input_tensor = test_input[0].unsqueeze(0)
    input_tensor.requires_grad = True
    target = test_input[1]

    # Run attribution:
    attributions = ### YOUR CODE HERE ###

    return attributions

Below we provide a function to visualize the output of integrated gradients. It calls `apply_integrated_gradients` to use the code you wrote above.

In [ ]:
from captum.attr import visualization as viz

def visualize_integrated_gradients(test_input, model):
    attr_ig = apply_integrated_gradients(test_input, model)

    # Transpose integrated gradients output
    attr_ig = np.transpose(attr_ig[0].cpu().detach().numpy(), (1, 2, 0))
    
    # Transpose and normalize original image:
    original_image = np.transpose((test_input[0].detach().numpy() * 0.5) + 0.5, (1, 2, 0))

    # This visualises the attribution of labels to pixels
    viz.visualize_image_attr(attr_ig, 
                             original_image, 
                             method="blended_heat_map",
                             sign="all",
                             show_colorbar=True, 
                             title="Overlayed Integrated Gradients",
                             fig_size=(3,3))

<div class="alert alert-info"><h4>
    Task 4.3: Visualize Integrated Gradients Examples</h4>

Call the `visualize_integrated_gradients` with...

* the "tainted" classifier on a "tainted" image of number 7
* the "clean" classifier on a "tainted" image of number 7
* the "tainted" classifier on a "clean" image of number 7
* of the "clean" classifier on a "clean" image of number 7
* of the "tainted" classifier on a "tainted" image of number 4
* of the "clean" classifier on a "tainted" image of number 4

You may need to search through the test dataset labels (`true_labels`) to find indices of 7s and 4s.
</div>

<div class="alert alert-block alert-warning"><h3>
    Questions</h3>
    <ol>
        <li>Where did the "tainted" model focus its attention on for the tainted "4" and "7"?</li>
        <li>Where did the "clean" model focus its attention on for the tainted "4" and "7"?</li>
        <li>How do these results help you interpret the confusion matrices? Were your predictions correct about why certain models did better or worse on certain digits?</li>
    </ol>
</div>

<div class="alert alert-block alert-success"><h3>
    Checkpoint 2</h3>
    <ol>
        Congrats on finishing the intergrated gradients task! Let us know on Element, and feel free to try other methods mentioned in the linked tutorial if you're interested.
    </ol>
</div>

## Part 5: Importance of using the right training data

Now we will move on from image classification to denoising, and show why it is particularly important to ensure that your training and test data are from the same distribution for these kinds of networks.

For this exercise, we will first train a simple CNN model to denoise MNIST images of digits, and then apply it to the Fashion MNIST to see what happens when the training and inference data are mismatched.


First, we will write a function to add noise to the MNIST dataset, so that we can train a model to denoise it.

In [ ]:
import torch

# A simple function to add noise to tensors:
def add_noise(tensor, power=1.5):
    return tensor * torch.rand(tensor.size()).to(tensor.device) ** power + 0.75*torch.randn(tensor.size()).to(tensor.device)


Next we will visualize a couple MNIST examples with and without noise.


In [ ]:
import matplotlib.pyplot as plt

# Let's visualise MNIST images with noise:
def show(index):
    plt.subplot(1,4,1)
    plt.imshow(train_dataset[index][0][0], cmap=plt.get_cmap('gray'))
    plt.subplot(1,4,2)
    plt.imshow(add_noise(train_dataset[index][0][0]), cmap=plt.get_cmap('gray'))
    plt.subplot(1,4,3)
    plt.imshow(train_dataset[index+1][0][0], cmap=plt.get_cmap('gray'))
    plt.subplot(1,4,4)
    plt.imshow(add_noise(train_dataset[index+1][0][0]), cmap=plt.get_cmap('gray'))
    plt.show()

# We pick 8 images to show:
for i in range(8):
    show(123*i)

### UNet model

Let's try denoising with a UNet, "CARE-style". As UNets and denoising implementations are not the focus of this exercise, we provide the model for you in the following cell. 

In [ ]:
# Adapted from https://discuss.pytorch.org/t/unet-implementation/426

import torch
from torch import nn
import torch.nn.functional as F


class UNet(nn.Module):
    def __init__(
        self,
        in_channels=1,
        n_classes=1,
        depth=3,
        wf=4,
        padding=True,
        batch_norm=False,
        up_mode='upsample',
    ):
        """
        Implementation of
        U-Net: Convolutional Networks for Biomedical Image Segmentation
        (Ronneberger et al., 2015)
        https://arxiv.org/abs/1505.04597
        Using the default arguments will yield the exact version used
        in the original paper
        Args:
            in_channels (int): number of input channels
            n_classes (int): number of output channels
            depth (int): depth of the network
            wf (int): number of filters in the first layer is 2**wf
            padding (bool): if True, apply padding such that the input shape
                            is the same as the output.
                            This may introduce artifacts
            batch_norm (bool): Use BatchNorm after layers with an
                               activation function
            up_mode (str): one of 'upconv' or 'upsample'.
                           'upconv' will use transposed convolutions for
                           learned upsampling.
                           'upsample' will use bilinear upsampling.
        """
        super(UNet, self).__init__()
        assert up_mode in ('upconv', 'upsample')
        self.padding = padding
        self.depth = depth
        prev_channels = in_channels
        self.down_path = nn.ModuleList()
        for i in range(depth):
            self.down_path.append(
                UNetConvBlock(prev_channels, 2 ** (wf + i), padding, batch_norm)
            )
            prev_channels = 2 ** (wf + i)

        self.up_path = nn.ModuleList()
        for i in reversed(range(depth - 1)):
            self.up_path.append(
                UNetUpBlock(prev_channels, 2 ** (wf + i), up_mode, padding, batch_norm)
            )
            prev_channels = 2 ** (wf + i)

        self.last = nn.Conv2d(prev_channels, n_classes, kernel_size=1)

    def forward(self, x):
        blocks = []
        for i, down in enumerate(self.down_path):
            x = down(x)
            if i != len(self.down_path) - 1:
                blocks.append(x)
                x = F.max_pool2d(x, 2)

        for i, up in enumerate(self.up_path):
            x = up(x, blocks[-i - 1])

        return self.last(x)


class UNetConvBlock(nn.Module):
    def __init__(self, in_size, out_size, padding, batch_norm):
        super(UNetConvBlock, self).__init__()
        block = []

        block.append(nn.Conv2d(in_size, out_size, kernel_size=3, padding=int(padding)))
        block.append(nn.ReLU())
        if batch_norm:
            block.append(nn.BatchNorm2d(out_size))

        block.append(nn.Conv2d(out_size, out_size, kernel_size=3, padding=int(padding)))
        block.append(nn.ReLU())
        if batch_norm:
            block.append(nn.BatchNorm2d(out_size))

        self.block = nn.Sequential(*block)

    def forward(self, x):
        out = self.block(x)
        return out


class UNetUpBlock(nn.Module):
    def __init__(self, in_size, out_size, up_mode, padding, batch_norm):
        super(UNetUpBlock, self).__init__()
        if up_mode == 'upconv':
            self.up = nn.ConvTranspose2d(in_size, out_size, kernel_size=2, stride=2)
        elif up_mode == 'upsample':
            self.up = nn.Sequential(
                nn.Upsample(mode='bilinear', scale_factor=2),
                nn.Conv2d(in_size, out_size, kernel_size=1),
            )

        self.conv_block = UNetConvBlock(in_size, out_size, padding, batch_norm)

    def center_crop(self, layer, target_size):
        _, _, layer_height, layer_width = layer.size()
        diff_y = (layer_height - target_size[0]) // 2
        diff_x = (layer_width - target_size[1]) // 2
        return layer[
            :, :, diff_y : (diff_y + target_size[0]), diff_x : (diff_x + target_size[1])
        ]

    def forward(self, x, bridge):
        up = self.up(x)
        crop1 = self.center_crop(bridge, up.shape[2:])
        out = torch.cat([up, crop1], 1)
        out = self.conv_block(out)

        return out

The training loop code is also provided here. It is similar to the code used to train the image classification model previously, but look it over to make sure there are no surprises.

In [ ]:
from tqdm.notebook import tqdm

def train_denoising_model(dataset, model, criterion, optimizer, history):
    
    # Puts model in 'training' mode:
    model.train()
    
    # Initialises progress bar:
    pbar = tqdm(total=len(dataset)//batch_size_train)
    for batch_idx, (image, target) in enumerate(train_loader):

        # add line here during Task 2.2
        
        # Zeroing gradients:
        optimizer.zero_grad()
        
        # Moves image to GPU memory:
        image = image.cuda()
        
        # Adds noise to make the noisy image:
        noisy = add_noise(image)
        
        # Runs model on noisy image:
        output = model(noisy)
        
        # Computes loss:
        loss = criterion(output, image)
        
        # Backpropagates gradients:
        loss.backward()
        
        # Optimises model parameters given the current gradients:
        optimizer.step()
        
        # appends loss history:
        history["loss"].append(loss.item())
        
        # updates progress bar:
        pbar.update(1)
    return history

Here we choose hyperparameters and initialize the model and data loaders.

In [ ]:
import torch.optim as optim
import torch

# Some hyper-parameters:
n_epochs = 5
batch_size_train = 64
batch_size_test = 1000

# Dictionary to store loss history:
history = {"loss": []}

# Model:
unet_model = UNet().cuda()

# Loss function:
criterion = F.mse_loss #mse_loss

# Optimiser:
optimizer = optim.Adam(unet_model.parameters(), lr=0.0005)

# Test loader:
test_loader = torch.utils.data.DataLoader(test_dataset,
  batch_size=batch_size_test, shuffle=True)

# Train loader:
train_loader = torch.utils.data.DataLoader(train_dataset,
  batch_size=batch_size_train, shuffle=True)

Finally, we run the training loop! This may take a little while - feel free to look more into attribution methods or chat with your neighbor about what will happen when we apply this model to the Fashion MNIST data.

In [ ]:
# Training loop:
for epoch in range(n_epochs):
    train_denoising_model(train_dataset, unet_model, criterion, optimizer, history)

As before, we will visualize the training loss. If all went correctly, it should decrease from around 1.0 to less than 0.2.

In [ ]:
# Let's look at the loss over time:
fig = plt.figure()
plt.plot(history["loss"], color='blue')
plt.legend(['Train Loss'], loc='upper right')
plt.xlabel('number of training examples seen')
plt.ylabel('mean squared error loss')

### Check denoising performance

We see that the training loss decreased, but let's apply the model to the test set to see how well it was able to recover the digits from the noisy images.

In [ ]:
def apply_denoising(image, model):
    # add batch and channel dimensions
    image = torch.unsqueeze(torch.unsqueeze(image, 0), 0)
    prediction = model(image)
    # remove batch and channel dimensions before returning
    return prediction.detach().cpu()[0,0]


In [ ]:
# Displays: ground truth, noisy, and denoised images
def visualize_denoising(model, dataset, index):
    orig_image = dataset[index][0][0]
    noisy_image = add_noise(orig_image)
    denoised_image = apply_denoising(noisy_image, model)
    plt.subplot(1,4,1)
    plt.imshow(orig_image, cmap=plt.get_cmap('gray'))
    plt.subplot(1,4,2)
    plt.imshow(noisy_image, cmap=plt.get_cmap('gray'))
    plt.subplot(1,4,3)
    plt.imshow(denoised_image, cmap=plt.get_cmap('gray'))
    
    plt.show()

# We pick 8 images to show:
for i in range(8):
    visualize_denoising(unet_model, test_dataset, 123*i)
    

<div class="alert alert-block alert-warning"><h3>
    Questions</h3>
    <ol>
        <li>Did the denoising net trained on MNIST work well on unseen test data?</li>
        <li>Do you think it would work well with other unseen data?</li>
    </ol>
</div>

### Apply trained model on 'wrong' data 

Apply the denoising model trained above to some example _noisy_ images derived from the Fashion-MNIST dataset.


### Load the Fashion MNIST dataset

Similar to the regular MNIST, we will use the pytorch FashionMNIST dataset. This was downloaded in the setup.sh script, so here we are just loading it into memory.

In [ ]:
fm_train_dataset = torchvision.datasets.FashionMNIST('./fashion_mnist', train=True, download=False,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ]))

fm_test_dataset = torchvision.datasets.FashionMNIST('./fashion_mnist', train=False, download=False,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ]))


<div class="alert alert-info"><h4>
    Task 5.1: Apply the Denoising Network to Fashion MNIST</h4>

Use the `apply_denoising` and/or `visualize_denoising` functions defined above to apply our denoising `unet_model` to the fashion mnist data. Visualize some results and take your time to interpret them.
</div>

In [ ]:
"""
Implement your solution here.
"""

<div class="alert alert-block alert-warning"><h3>
    Questions</h3>
    <ol>
        <li>Did the denoising net trained on MNIST work well on Fashion MNIST?</li>
        <li>Why or why not do you think?</li>
        <li>What kind of scenarios might this pitfall happen in the real world?</li>
        <li>Any idea on how to make the network more generalized?</li>
    </ol>
</div>

<div class="alert alert-block alert-success"><h3>
    Checkpoint 3</h3>
    <ol>
        Congrats on reaching the final checkpoint! Let us know on Element, and we'll discuss the questions once reaching critical mass.
    </ol>
</div>

<div class="alert alert-block alert-info"><h3>
    Extra credits</h3>
    <ol>
        <li>Decimate training data: In the code above, add filter every training batch so that it only contains images of class "1". Skip the batch if there are no "1"s in the batch.</li>
        <li>Redo the exercises but swap MNIST with Fashion-MNIST</li>
        <li>Don't hesitate to try things, explore the behaviour of the network... For example, you can also try reducing the size of the training data, can you cause the network to overfit? How would you cause the network to underfit?</li>
    </ol>
</div>